In [1]:
## Utterance level comics, FT, unilabel

In [1]:
from unsloth.chat_templates import get_chat_template

import torch
import json_repair
import pandas as pd

from tqdm import tqdm
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments

from unsloth import FastLanguageModel, is_bfloat16_supported

from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    #model_name="unsloth/Meta-Llama-3.1-8B-Instruct-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 2080 Ti. Num GPUs = 1. Max memory: 10.747 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=32,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing=True
)

Unsloth 2025.3.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### data

In [4]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_dataset.csv")

In [5]:
df

,file_name,page_nr,panel_nr,balloon_nr,utterance,raw_annotation,raw_emotion,raw_speaker_id,emotion,speaker_id,split
0,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,1,DID YOU HAVE TO ELECTROCUTE HER SO HARD?,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE3-SA0-SU5-JO0,ID-1,TRAIN
1,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,2,IT'S NOT LIKE I HAVE DIFFERENT SETTINGS.,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU5-JO0,ID-2,TRAIN
2,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,3,YOU'RE ELECTROCUTIONER. IT'S YOUR WHOLE THING....,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE2-SA0-SU0-JO0,ID-1,TRAIN
3,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,3,1,"OH, HEY. I THINK SHE'S AWAKE.",2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU4-JO0,ID-2,TRAIN
4,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,4,1,"WELCOME BACK, MADAM MAYOR. BLOCKBUSTER IS PRET...",2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN3-DI0-FE0-SA0-SU0-JO0,ID-1,TRAIN
...,...,...,...,...,...,...,...,...,...,...,...
7124,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,2,SHE WOULDN'T DO THAT TO US. WE TALKED FOR A LO...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN
7125,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,3,… I KNOW HER.,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN
7126,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,4,1,"UH, GUYS…",2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:JUANITA...,AN0-DI0-FE3-SA0-SU4-JO0,JUANITA SANCHEZ,TRAIN
7127,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,22,1,1,PUT YOUR WEAPONS DOWN AND PUT YOUR HANDS IN TH...,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:ID- 2,AN4-DI0-FE0-SA0-SU0-JO0,ID- 2,TRAIN


In [6]:
import pandas as pd

def map_emotions(row):
    
    emotion_str = row.emotion
    emotions = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]
    
    try:
        parts = emotion_str.split('-')
        # Extract valid emotion and value pairs
        emotion_values = []
        for i, part in enumerate(parts):
            if len(part) > 2 and part[2:].isdigit():  # Ensure valid integer after the abbreviation
                value = int(part[2:])
                if value > 0:
                    emotion_values.append((i, value))
        
        if not emotion_values:
            return None  # No non-zero values

        # Find the emotion with the largest value, choose the first if tied
        max_emotion_index = max(emotion_values, key=lambda x: (x[1], -x[0]))[0]
        return emotions[max_emotion_index]

    except Exception as e:
        print(f"Error processing '{emotion_str}': {e}")
        return None

In [7]:
df['emotion_label'] = df.apply(lambda row: map_emotions(row), axis=1) # type: ignore

In [8]:
df

,file_name,page_nr,panel_nr,balloon_nr,utterance,raw_annotation,raw_emotion,raw_speaker_id,emotion,speaker_id,split,emotion_label
0,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,1,DID YOU HAVE TO ELECTROCUTE HER SO HARD?,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE3-SA0-SU5-JO0,ID-1,TRAIN,Surprise
1,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,2,IT'S NOT LIKE I HAVE DIFFERENT SETTINGS.,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU5-JO0,ID-2,TRAIN,Surprise
2,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,3,YOU'RE ELECTROCUTIONER. IT'S YOUR WHOLE THING....,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE2-SA0-SU0-JO0,ID-1,TRAIN,Fear
3,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,3,1,"OH, HEY. I THINK SHE'S AWAKE.",2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU4-JO0,ID-2,TRAIN,Surprise
4,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,4,1,"WELCOME BACK, MADAM MAYOR. BLOCKBUSTER IS PRET...",2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN3-DI0-FE0-SA0-SU0-JO0,ID-1,TRAIN,Anger
...,...,...,...,...,...,...,...,...,...,...,...,...
7124,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,2,SHE WOULDN'T DO THAT TO US. WE TALKED FOR A LO...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN,Sadness
7125,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,3,… I KNOW HER.,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN,Sadness
7126,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,4,1,"UH, GUYS…",2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:JUANITA...,AN0-DI0-FE3-SA0-SU4-JO0,JUANITA SANCHEZ,TRAIN,Surprise
7127,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,22,1,1,PUT YOUR WEAPONS DOWN AND PUT YOUR HANDS IN TH...,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:ID- 2,AN4-DI0-FE0-SA0-SU0-JO0,ID- 2,TRAIN,Anger


In [9]:
df_train = df[df.split == "TRAIN"].reset_index(drop=True)

In [10]:
df.emotion_label.value_counts()

emotion_label
Anger       1972
Joy         1352
Sadness     1169
Fear        1105
Surprise    1001
Disgust       92
Name: count, dtype: int64

In [11]:
### Prepare Prompts

In [12]:
def build_instruction():
    
    emotion_classes = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comics dialogue emotion interpretation.

INPUT:
- Single utterance to analyze

TASK:
Identify ONLY ONE applicable emotions from: {formatted_classes}

RULES:
1. Use ONLY the listed labels
2. Output must be a JSON with key "emotion"
4. Multiple emotions are NOT allowed
5. No explanations, only JSON output

"""
    return instruction

In [13]:
instruction = build_instruction()

In [14]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df_train.iterrows():
        
        sys_msg = {'role': 'system', 'content': instruction}
        user_msg = {'role': 'user', 'content': row.utterance}
        assistant_msg = {'role': 'assistant', 'content': f'{{"emotion": {row["emotion_label"]}}}'}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [15]:
comics_dataset = []

for i in range(len(sys_msg_l)):
    
    comics_dataset.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [16]:
def fix_comics_dataset(comics_dataset):
    fixed_comics_dataset = []
    for conversation in comics_dataset:
        fixed_conversation = []
        for message in conversation:
            if isinstance(message['content'], list):  # If the 'value' is a list of emotions
                message['content'] = ', '.join(message['content'])  # Join the list into a string
            fixed_conversation.append(message)
        fixed_comics_dataset.append(fixed_conversation)
    return fixed_comics_dataset

In [17]:
fixed_comics_dataset = fix_comics_dataset(comics_dataset)

In [18]:
dataset = Dataset.from_dict({
    'conversations': fixed_comics_dataset
})

In [19]:
dataset[0]['conversations']

[{'content': '### Emotion Analysis Expert Role\n\nYou are an advanced emotion analysis expert specializing in comics dialogue emotion interpretation.\n\nINPUT:\n- Single utterance to analyze\n\nTASK:\nIdentify ONLY ONE applicable emotions from: "Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"\n\nRULES:\n1. Use ONLY the listed labels\n2. Output must be a JSON with key "emotion"\n4. Multiple emotions are NOT allowed\n5. No explanations, only JSON output\n\n',
  'role': 'system'},
 {'content': 'DID YOU HAVE TO ELECTROCUTE HER SO HARD?', 'role': 'user'},
 {'content': '{"emotion": Surprise}', 'role': 'assistant'}]

In [20]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template_comics(examples):
    messages = examples["conversations"]
    #messages = examples['input'] + examples['output']
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.


In [21]:
comics_dataset = dataset.map(apply_template_comics, batched=True)

Map:   0%|          | 0/5803 [00:00<?, ? examples/s]

In [22]:
def split_dataset(dataset, train_ratio=0.8):
    train_test = dataset.train_test_split(test_size=1 - train_ratio)
    return train_test

dataset_split = split_dataset(comics_dataset)

In [23]:
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

In [24]:
print(eval_dataset[360]['text'])

<|im_start|>system
### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comics dialogue emotion interpretation.

INPUT:
- Single utterance to analyze

TASK:
Identify ONLY ONE applicable emotions from: "Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"

RULES:
1. Use ONLY the listed labels
2. Output must be a JSON with key "emotion"
4. Multiple emotions are NOT allowed
5. No explanations, only JSON output

<|im_end|>
<|im_start|>user
AND HOW WOULD YOU PROPOSE WE DO THAT?<|im_end|>
<|im_start|>assistant
{"emotion": Surprise}<|im_end|>



In [25]:
### Training

In [26]:
OUTPUT_DIR = "/Utilisateurs/umushtaq/emotion_analysis_comics/unilabel_comics_model"

In [27]:
args=TrainingArguments(
    
        learning_rate=5e-5,
        lr_scheduler_type="cosine",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        num_train_epochs=5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=50,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        
        eval_strategy="steps",  # Run evaluation during training (can also use "epoch")
        eval_steps=50,  # Perform evaluation every 50 steps
        save_strategy="steps",  # Save the model every few steps
        save_steps=50,  # Save every 200 steps
        load_best_model_at_end=True,
    
        output_dir=OUTPUT_DIR,
        seed=0,
    )

In [28]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Replace with your train dataset
    eval_dataset=eval_dataset, 
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=args,
)

Map (num_proc=2):   0%|          | 0/4642 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1161 [00:00<?, ? examples/s]

In [29]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,642 | Num Epochs = 5 | Total steps = 725
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 2 x 1) = 32
 "-____-"     Trainable parameters = 83,886,080/4,624,486,400 (1.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,0.708800,0.295504
100,0.290100,0.286203
150,0.278400,0.284673
200,0.248400,0.286526
250,0.241300,0.286040
300,0.229700,0.315696
350,0.182500,0.308536
400,0.185100,0.306367
450,0.168700,0.369348
500,0.123900,0.353081


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=725, training_loss=0.2152244893435774, metrics={'train_runtime': 6585.5513, 'train_samples_per_second': 3.524, 'train_steps_per_second': 0.11, 'total_flos': 1.9589158076925542e+17, 'train_loss': 0.2152244893435774, 'epoch': 4.969072164948454})

In [30]:
### inference

In [31]:
model = FastLanguageModel.for_inference(model)

In [32]:
df_test = df[df.split == "TEST"].reset_index(drop=True)

In [33]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df_test.iterrows():
        
        sys_msg = {'role': 'system', 'content': instruction}
        user_msg = {'role': 'user', 'content': row.utterance}
        assistant_msg = {'role': 'assistant', 'content': ""}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [34]:
test_messages = []

for i in range(len(sys_msg_l)):
    
    test_messages.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [35]:
raw_outputs = []

for message in tqdm(test_messages):
    
    inputs = tokenizer.apply_chat_template(
    message,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt",
).to("cuda")
    #print(inputs)
    #break
    
    output = model.generate(**inputs, max_new_tokens=128)[0]
    
    input_length = inputs.input_ids.shape[1]
    generated_tokens = output[input_length:]
    
    decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
    #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
    raw_outputs.append(decoded_output)
    #break

  0%|          | 0/1326 [00:00<?, ?it/s]

100%|██████████| 1326/1326 [07:35<00:00,  2.91it/s]


In [36]:
### post processing

In [37]:
raw_outputs

['{"emotion": Anger}',
 '{"emotion": Joy}',
 '{"emotion": Joy}',
 '{"emotion": Fear}',
 '{"emotion": Surprise}',
 '{"emotion": Joy}',
 '{"emotion": Anger}',
 '{"emotion": Fear}',
 '{"emotion": Fear}',
 '{"emotion": Sadness}',
 '{"emotion": Sadness}',
 '{"emotion": Anger}',
 '{"emotion": Sadness}',
 '{"emotion": Joy}',
 '{"emotion": Anger}',
 '{"emotion": None}',
 '{"emotion": Surprise}',
 '{"emotion": Sadness}',
 '{"emotion": Joy}',
 '{"emotion": None}',
 '{"emotion": Surprise}',
 '{"emotion": Joy}',
 '{"emotion": Anger}',
 '{"emotion": Surprise}',
 '{"emotion": Anger}',
 '{"emotion": Joy}',
 '{"emotion": Fear}',
 '{"emotion": Sadness}',
 '{"emotion": Fear}',
 '{"emotion": Joy}',
 '{"emotion": Anger}',
 '{"emotion": Surprise}',
 '{"emotion": Joy}',
 '{"emotion": Sadness}',
 '{"emotion": Fear}',
 '{"emotion": Joy}',
 '{"emotion": Joy}',
 '{"emotion": Joy}',
 '{"emotion": Joy}',
 '{"emotion": Sadness}',
 '{"emotion": Sadness}',
 '{"emotion": Surprise}',
 '{"emotion": Anger}',
 '{"emotion

In [38]:
predictions = [json_repair.loads(e) for e in raw_outputs]

In [39]:
predictions

[{'emotion': 'Anger'},
 {'emotion': 'Joy'},
 {'emotion': 'Joy'},
 {'emotion': 'Fear'},
 {'emotion': 'Surprise'},
 {'emotion': 'Joy'},
 {'emotion': 'Anger'},
 {'emotion': 'Fear'},
 {'emotion': 'Fear'},
 {'emotion': 'Sadness'},
 {'emotion': 'Sadness'},
 {'emotion': 'Anger'},
 {'emotion': 'Sadness'},
 {'emotion': 'Joy'},
 {'emotion': 'Anger'},
 {'emotion': 'None'},
 {'emotion': 'Surprise'},
 {'emotion': 'Sadness'},
 {'emotion': 'Joy'},
 {'emotion': 'None'},
 {'emotion': 'Surprise'},
 {'emotion': 'Joy'},
 {'emotion': 'Anger'},
 {'emotion': 'Surprise'},
 {'emotion': 'Anger'},
 {'emotion': 'Joy'},
 {'emotion': 'Fear'},
 {'emotion': 'Sadness'},
 {'emotion': 'Fear'},
 {'emotion': 'Joy'},
 {'emotion': 'Anger'},
 {'emotion': 'Surprise'},
 {'emotion': 'Joy'},
 {'emotion': 'Sadness'},
 {'emotion': 'Fear'},
 {'emotion': 'Joy'},
 {'emotion': 'Joy'},
 {'emotion': 'Joy'},
 {'emotion': 'Joy'},
 {'emotion': 'Sadness'},
 {'emotion': 'Sadness'},
 {'emotion': 'Surprise'},
 {'emotion': 'Anger'},
 {'emotion'

In [40]:
predictions = [elem['emotion'] for elem in predictions]

In [41]:
df_test

,file_name,page_nr,panel_nr,balloon_nr,utterance,raw_annotation,raw_emotion,raw_speaker_id,emotion,speaker_id,split,emotion_label
0,QC copy - 1643 - 23 American Vampire vol. 4 - ...,1,1,1,"TIME TO FACE OUR FEARS, PEOPLE…",2024-08-25 - SyimykRasulov\nFeeling:Neutral\n\...,2024-08-25 - SyimykRasulov\nFeeling:Neutral,\n2024-08-25 - SyimykRasulov\nSpokenBy:unknown,Neutral,unknown,TEST,None
1,QC copy - 1643 - 23 American Vampire vol. 4 - ...,1,1,3,IT'S TIME TO LIMBO!,2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-08-25 - SyimykRasulov\nSpokenBy:ID- 1,AN0-DI0-FE0-SA0-SU0-JO3,ID- 1,TEST,Joy
2,QC copy - 1643 - 23 American Vampire vol. 4 - ...,1,2,1,"HOW LOW, CAN YOU GO?! HOW LOW, CAN YOU GO?!",2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-08-25 - SyimykRasulov\nSpokenBy:ID- 2,AN0-DI0-FE0-SA0-SU2-JO3,ID- 2,TEST,Joy
3,QC copy - 1643 - 23 American Vampire vol. 4 - ...,1,2,2,HOW LOW--,2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-08-25 - SyimykRasulov\nSpokenBy:ID- 2,AN0-DI0-FE0-SA0-SU2-JO3,ID- 2,TEST,Joy
4,QC copy - 1643 - 23 American Vampire vol. 4 - ...,1,2,3,WHAT IN…,2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-08-25 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-08-25 - SyimykRasulov\nSpokenBy:ID- 3,AN0-DI0-FE3-SA0-SU4-JO0,ID- 3,TEST,Surprise
...,...,...,...,...,...,...,...,...,...,...,...,...
1321,QC copy - 1514 - 15 DC contra Vampiros 11.xlsx,22,2,3,DAMIAN! YOU'RE ALIVE!,2024-08-04 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-04 - aselermekova20\nFeeling:AN0-DI0-F...,\n2024-09-11 - aidaraliev12345\nSpokenBy:Batgirl,AN0-DI0-FE0-SA0-SU5-JO0,Batgirl,TEST,Surprise
1322,QC copy - 1514 - 15 DC contra Vampiros 11.xlsx,22,2,4,NO. HE'S NOT. HE'S A VAMPIRE.,2024-08-04 - aselermekova20\nFeeling:Neutral\n...,2024-08-04 - aselermekova20\nFeeling:Neutral,\n2024-09-11 - aidaraliev12345\nSpokenBy:Frank...,Neutral,Frankenstein,TEST,None
1323,QC copy - 1514 - 15 DC contra Vampiros 11.xlsx,22,3,1,WHOA. WHOSE BIG BRAIN DID THEY PUT IN THAT HEA...,2024-08-04 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-04 - aselermekova20\nFeeling:AN0-DI0-F...,\n2024-09-11 - aidaraliev12345\nSpokenBy:Damian,AN0-DI0-FE0-SA0-SU5-JO3,Damian,TEST,Surprise
1324,QC copy - 1514 - 15 DC contra Vampiros 11.xlsx,22,3,2,SPARE PARTS IS RIGHT. I AM A VAMPIRE.,2024-08-04 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-04 - aselermekova20\nFeeling:AN0-DI0-F...,\n2024-09-11 - aidaraliev12345\nSpokenBy:Damian,AN0-DI0-FE0-SA0-SU2-JO3,Damian,TEST,Joy


In [42]:
grounds = df_test.emotion_label

In [43]:
grounds

0           None
1            Joy
2            Joy
3            Joy
4       Surprise
          ...   
1321    Surprise
1322        None
1323    Surprise
1324         Joy
1325         Joy
Name: emotion_label, Length: 1326, dtype: object

In [44]:
none_indices = [i for i, value in enumerate(grounds) if value is None]

In [45]:
grounds_l = [grounds[i] for i in range(len(grounds)) if i not in none_indices]
predictions_l = [predictions[i] for i in range(len(predictions)) if i not in none_indices]

In [46]:
len(grounds_l), len(predictions_l)

(1217, 1217)

In [47]:
grounds_l

['Joy',
 'Joy',
 'Joy',
 'Surprise',
 'Anger',
 'Anger',
 'Fear',
 'Fear',
 'Joy',
 'Sadness',
 'Fear',
 'Sadness',
 'Sadness',
 'Sadness',
 'Surprise',
 'Anger',
 'Surprise',
 'Fear',
 'Disgust',
 'Surprise',
 'Surprise',
 'Sadness',
 'Fear',
 'Fear',
 'Surprise',
 'Surprise',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Sadness',
 'Joy',
 'Anger',
 'Surprise',
 'Anger',
 'Sadness',
 'Surprise',
 'Surprise',
 'Surprise',
 'Surprise',
 'Fear',
 'Surprise',
 'Surprise',
 'Fear',
 'Surprise',
 'Surprise',
 'Surprise',
 'Surprise',
 'Fear',
 'Surprise',
 'Surprise',
 'Sadness',
 'Sadness',
 'Surprise',
 'Fear',
 'Fear',
 'Sadness',
 'Joy',
 'Disgust',
 'Disgust',
 'Joy',
 'Joy',
 'Sadness',
 'Fear',
 'Surprise',
 'Surprise',
 'Joy',
 'Joy',
 'Sadness',
 'Joy',
 'Sadness',
 'Fear',
 'Sadness',
 'Sadness',
 'Sadness',
 'Anger',
 'Fear',
 'Fear',
 'Fear',
 'Fear',
 'Fear',
 'Fear',
 'Fear',
 'Sadness',
 'Sadness',
 'Anger',
 'Anger',
 'Anger',
 'Sadness',
 'Anger',
 'Sadness

In [48]:
print(classification_report(grounds_l, predictions_l, digits=3))

              precision    recall  f1-score   support

       Anger      0.526     0.526     0.526       365
     Disgust      0.000     0.000     0.000        18
        Fear      0.355     0.544     0.430       182
         Joy      0.507     0.494     0.500       235
        None      0.000     0.000     0.000         0
     Sadness      0.480     0.392     0.431       217
    Surprise      0.509     0.420     0.460       200

    accuracy                          0.473      1217
   macro avg      0.340     0.339     0.335      1217
weighted avg      0.478     0.473     0.471      1217



/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi